In [30]:
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer
x = ['which book is this','this is book and this is math']

In [2]:
cv = CountVectorizer()
count = cv.fit_transform(x)
count.toarray()

array([[0, 1, 1, 0, 1, 1],
       [1, 1, 2, 1, 2, 0]], dtype=int64)

In [4]:
cv.get_feature_names_out()

array(['and', 'book', 'is', 'math', 'this', 'which'], dtype=object)

In [56]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [7]:
bow = pd.DataFrame(count.toarray(),columns =['and', 'book', 'is', 'math', 'this', 'which'] )

In [8]:
bow

,and,book,is,math,this,which
0,0,1,1,0,1,1
1,1,1,2,1,2,0


In [9]:
bow.shape

(2, 6)

In [10]:
enumerate(bow)

In [14]:
tf = bow.copy()
for index,row in enumerate(tf.iterrows()):
    for col in row[1].index:
        tf.loc[index,col] = tf.loc[index,col]/sum(row[1])

In [15]:
tf

,and,book,is,math,this,which
0,0.000000,0.250000,0.250000,0.000000,0.250000,0.25
1,0.142857,0.142857,0.285714,0.142857,0.285714,0.00


In [16]:
bow

,and,book,is,math,this,which
0,0,1,1,0,1,1
1,1,1,2,1,2,0


In [20]:
bb = bow.astype('bool')
bb['and'].sum()

1

In [22]:
cols = bb.columns
cols

Index(['and', 'book', 'is', 'math', 'this', 'which'], dtype='object')

In [23]:
nz = []
for col in cols:
    nz.append(bb[col].sum())

In [24]:
nz

[1, 2, 2, 1, 2, 1]

In [27]:
idf = []
N = 2
for index, col in enumerate(cols):
    idf.append(np.log((N+1)/(nz[index]+1))+1)

In [28]:
idf

[1.4054651081081644, 1.0, 1.0, 1.4054651081081644, 1.0, 1.4054651081081644]

In [29]:
x

['which book is this', 'this is book and this is math']

In [31]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(x)

In [33]:
print(X.toarray())

[[0.         0.44832087 0.44832087 0.         0.44832087 0.63009934]
 [0.39054766 0.27787788 0.55575576 0.39054766 0.55575576 0.        ]]


In [34]:
print(tfidf.idf_)

[1.40546511 1.         1.         1.40546511 1.         1.40546511]


In [62]:
data = "C:\\Users\\Aditya Shakya\\OneDrive\\Desktop\\datasets\\email_spam.csv"

In [63]:
df = pd.read_csv(data)

In [64]:
df.shape

(5572, 2)

In [65]:
df.head()

,Category,Message
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


In [66]:
df['Category'].value_counts()

ham     4825
spam     747
Name: Category, dtype: int64

In [67]:
df['Category'] = df['Category'].apply(lambda x : 1 if x=='ham' else 0)


In [68]:
df['Category']

0       1
1       1
2       0
3       1
4       1
       ..
5567    0
5568    1
5569    1
5570    1
5571    1
Name: Category, Length: 5572, dtype: int64

In [69]:
df['length'] = df['Message'].apply(lambda x : len(x))

In [70]:
df.head()

,Category,Message,length
0,1,"Go until jurong point, crazy.. Available only ...",111
1,1,Ok lar... Joking wif u oni...,29
2,0,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,1,U dun say so early hor... U c already then say...,49
4,1,"Nah I don't think he goes to usf, he lives aro...",61


In [72]:
ham = df[df['Category']==1]
spam = df[df['Category']==0]

In [78]:
spam.shape

(747, 3)

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,confusion_matrix
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer,TfidfTransformer,TfidfVectorizer

In [120]:
tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df['Message'])
X = X.toarray()

In [86]:
y = df['Category'].values

### Random Forest

In [87]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2, random_state=42)

In [88]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(4457, 8709)
(1115, 8709)
(4457,)
(1115,)


In [89]:
clf = RandomForestClassifier(n_estimators=100, criterion = 'gini',n_jobs=2,min_samples_leaf=2)

In [90]:
model = clf.fit(X_train,y_train)

In [92]:
y_predict = model.predict(X_test)

In [101]:
print(accuracy_score(y_test,y_predict))

0.9757847533632287


In [100]:
print(confusion_matrix(y_test,y_predict))

[[122  27]
 [  0 966]]


In [99]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       1.00      0.82      0.90       149
           1       0.97      1.00      0.99       966

    accuracy                           0.98      1115
   macro avg       0.99      0.91      0.94      1115
weighted avg       0.98      0.98      0.97      1115



### SVM

In [102]:
sv = SVC(C = 1, kernel='sigmoid')

In [103]:
model_svm = sv.fit(X_train,y_train)

In [104]:
y_predict_sv = model.predict(X_test)

In [106]:
accuracy_score(y_test,y_predict)

0.9757847533632287

In [108]:
print(classification_report(y_test,y_predict))

              precision    recall  f1-score   support

           0       1.00      0.82      0.90       149
           1       0.97      1.00      0.99       966

    accuracy                           0.98      1115
   macro avg       0.99      0.91      0.94      1115
weighted avg       0.98      0.98      0.97      1115



In [109]:
print(confusion_matrix(y_test,y_predict))

[[122  27]
 [  0 966]]
